In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# 设置超参数
input_size  = 28 * 28   # MNIST数据集的图片大小为28x28
hidden_size = 512       # 隐藏层的大小
output_size = 10        # 分类10个类别（0-9）
batch_size  = 64        # 批处理大小
learning_rate = 0.001   # 学习率
epochs = 5              # 遍历数据集的次数

# 加载训练集和测试集
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

# 定义神经网络结构
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(-1, input_size)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# 实例化神经网络，定义损失函数和优化器
model = NeuralNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练神经网络
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 测试神经网络
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

# 开始训练和测试
for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.351524
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.250353
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.185336
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.060947
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.114748
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.055184
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.185889
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.226176
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.157013
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.127799

Test set: Average loss: 0.0014, Accuracy: 9750/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.091385
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.113465
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.058925
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.090337
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.089595
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.022185
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.074497
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.278713
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.096981
T

In [8]:
import torch
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))  # 获取设备 0 的名称
#torch.cuda.get_device_name(1)  # 获取设备 0 的名称
torch.cuda.is_available()

0
NVIDIA GeForce GTX 1650


True